<h1 align="center"> Separation of variables  </h1>
<h3 align="center">A symbolic algorithm for the maxima CAS </br> Nijso Beishuizen</h3>


We discuss a simple but effective implementation of a method to separate variables. Separation of variables is a well-known and much-used technique to solve ordinary and partial differential equations. 

An expression $F(x,y)$ with the 2 independent variables $x$ and $y$ is separable when $\frac{\partial F}{\partial x}/F = \frac{F_x}{F}$ depends only on $x$, see <a href='#ref:viazminsky'>[1]</a>, <a href='#ref:cid'>[1]</a>. This is easy to see because if $F$ is separable, it can be written as $F=f(x)\cdot R(y)$. If on both sides we take the derivative with respect to $x$ and divide by $F$ we obtain: $\frac{F_x}{F} = \frac{f_x}{f}$. Since we know that the right-hand side is a function of only x, the left-hand side must be as well.
The separated function $f(x)$ can then be found by solving the differential equation:

$|f(x)| = e^{\int(F_x/Fdx)}$ (1)

The function $g(y)$ can be found in a similar way, e.g. by computing:

$|g(y)| = e^{\int(F_y/Fdy)}$, (2)

or by using $g(y) = F(x,y)/f(x)$. Note that the latter approach might be computationally more costly because we have to properly factor out the x-dependency. With the above aproach eqs. (1)-(2), we lose any constant factors, e.g. when $F(x,y)=C \cdot f(x) \cdot g(y)$, we have lost $C$. We can find $C$ during the evaluation of our results: we check that 
$C = \frac{F(x,y)}{f(x)\cdot g(y)}$ is free of $x$ and $y$.

 
The maxima implementation for separation of variables has been implemented as follows:


# Function and variable index

<u>Function:</u> <b>separable</b>(expr,x,y,[options])

If the input expression is a separable function in the independent variables $x$ and $y$, e.g. the expression can be written as $C\cdot f(x)\cdot g(y)$, with $C$ a constant independent of $x,y$, then separable(expr,x,y) returns a list of the form $[f(x), C\cdot g(y)]$, or false when the expression is not separable.
When the optional input option *'splitConstant=true* is set, the constant factor will be separated as well and a list $[f(x),g(y),C]$ will be returned.


<u>Function:</u> <b>isSeparable</b>(expr,x,y)

If the input expression can be written as a separable expression $f(x)\cdot g(y)$, return true, otherwise return false.

<u>Function:</u> <b>constant_factors</b>(expr,varlist)

If the input expression can be written as $C\cdot F(x,y)$, with $C$ an expression free of the variables in the list varlist, *constant_factors* will return the constant, or 1.


# Example
For some expressions is it not immediately clear from inspection that they are separable. For instance the expression

$e^{x^2+y^2}(\cos(x+y) + \cos(x-y))$ 

can be rewritten as the product of separable functions: 

$e^{x^2} \cdot e^{y^2} \cdot 2\cos(x) \cos(y)$

The maxima implementation of the above algorithm recognizes this without relying on pattern matching to recognize this separation.  

In [24]:
kill(all);batch("~/mathematics/maxima_files/separable.mac");


read and interpret /home/nijso/mathematics/maxima_files/separable.mac


(%i2) batch("/home/nijso/mathematics/maxima_files/ode_extra.mac")


read and interpret /home/nijso/mathematics/maxima_files/ode_extra.mac


(%i3) DEBUGFLAG:1

(%i4) MAX_LENGTH_FOR_SIMPLIFICATION:30000

...

(%i30) isSeparable(_F,_x,_y):=block([_F1,_P,_Q,_L,_SP,_SQ,_dF,_S],
                   _F1:ratsimp(_F),
                   if freeof(_x,_y,_F1)
                       then (dprint(5,"separable: expression is C"),
                             return(true)),
                   if freeof(_x,_F1)
                       then (dprint(5,"separable: expression is f(y)"),
                             return(true)),
                   if freeof(_y,_F1)
                       then (dprint(5,"separable: expression is f(x)"),
                             return(true)),_P:num(_F1),_Q:denom(_F1),
                   _ratvars:showratvars(_F1),
                   _L:sublist(_ratvars,
                              lambda([i],not freeof(sqrt,dispform(i,all)))),
                   _L:map(args,_L),
                   _L:sublist(_L,
                              lambda([i],
                                     not freeof(_x,i) and not freeof(_y,i))),
                   _L:sublist(_L,lambda([i],op(i[1]) = "+")

(%i31) hasNegativeSign(expr):=block([],if atom(expr) then return(false),
                       if op(expr) = "-" then return(true) else return(false))

(%i32) constant_factors(_expr,_varlist):=block(
                        [inflag:true,_constantfactors:1,_oldratvars,
                         _substlist],
                        if not listp(_varlist) then _varlist:[_varlist],
                        _varlist:append(_varlist,map(op,dependencies)),
                        _oldratvars:ratvars,ratvars:_varlist,
                        _expr:ratsimp(_expr),
                        if lfreeof(_varlist,_expr) then _constantfactors:_expr
                            else (if not mapatom(_expr) and op(_expr) = "*"
                                      then _constantfactors
                                      :xreduce("*",
                                               listify(
                                                subset(setify(args(_expr)),
                                                       lambda([_u],
                                                              lfreeof(
                                                      

(%o0)                                done

(%o34)        /home/nijso/mathematics/maxima_files/separable.mac

In [15]:
expr1 : 6*e^(x^2+y^2)*(2*cos(x+y) + 2*cos(x-y));

                       2    2
                      y  + x
(%o69)             6 e        (2 cos(y + x) + 2 cos(y - x))

In [16]:
separable(expr1,x,y);

                             2              2
                            x              y
(%o70)                    [e   cos(x), 24 e   cos(y)]

The constant factor is always added to the second term $g(y)$. We can also separate the constant by using the optional command 'splitConstant=true. The constant factor is then added to the list.

In [17]:
expr2 : separable(expr1,x,y,'splitConstant=true);

                            2           2
                           x           y
(%o71)                   [e   cos(x), e   cos(y), 24]

We can also call the routine responsible for the separation of the constant factor independently:

In [18]:
constant_factors(5*c*x+5*c,[x]);

(%o72)                                5 c

constant_factors also tries to find a shared minus sign. If all terms in an expression of the form a+b+c+d have a minus sign, e.g. expr: -a-b-c-d, then the minus sign will also be factored out.

In [19]:
constant_factors(-5*c*x-5*c,[x]);

(%o73)                               - 5 c

Sometimes we just want to know if an expression is separable or not. We can then skip some internal computations.

In [20]:
isSeparable(x*y,x,y);

(%o74)                               true

# Separation of variables for an ODE

We can use separation of variables to solve ordinary differential equations. For the general separable ode:

In [21]:
ode:'diff(y,x)=f(x)*g(y);

                                dy
(%o75)                          -- = f(x) g(y)
                                dx

In [22]:
S:separable(rhs(ode),x,y);

(%o76)                           [f(x), g(y)]

In [23]:
sol:integrate(1/S[2],y)=integrate(S[1],x);

                             /           /
                             [  1        [
(%o77)                       I ---- dy = I f(x) dx
                             ] g(y)      ]
                             /           /

# Bibliography

[1] <a id='ref:viazminsky'></a> C.P. Viazminsky, On separation of variables, arXiv:math/0210167 (https://arxiv.org/pdf/math/0210167.pdf)

[2] <a id='ref:cid'></a> J.A. Cid, A simple method to find out when and ordinary differential equation is separable,  International Journal of Mathematical Education in Science and Technology Vol. 40 , Iss. 5,2009 (http://www4.ujaen.es/~angelcid/Archivos/Papers/IJMEST.pdf)